# GEM

$$
\mathcal{L} = 
$$

In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
from tqdm import tqdm

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training import GEM
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    confusion_matrix_metrics,
    forgetting_metrics,
    timing_metrics,
    cpu_usage_metrics,
    gpu_usage_metrics,
    disk_usage_metrics,
    ram_usage_metrics,
    loss_metrics,
    class_accuracy_metrics,
    accuracy_matrix_metrics
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.plugins import WeightStoragePlugin

# Constants

In [5]:
TORCH_DEVICE = "cuda" if cuda.is_available() else "cpu"
N_EXPERIENCES = 20
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = f"class_saturation/{N_EXPERIENCES}"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/gem.pkl"

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
scenario = PermutedMNIST(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

In [8]:
best_hyperparameters = {
    'patterns_per_exp': 2,
    'memory_strength': 0.5
}

## Evaluation metrics definition

In [9]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    loss_metrics(minibatch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(minibatch=True, experience=True, stream=True),
    gpu_usage_metrics(experience=True, stream=True, gpu_id=TORCH_DEVICE),
    disk_usage_metrics(experience=True, stream=True),
    ram_usage_metrics(experience=True, stream=True),
    class_accuracy_metrics(experience=True, stream=True),
    accuracy_matrix_metrics(),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    confusion_matrix_metrics(stream=True, num_classes=scenario.n_classes),
    
    loggers=[InteractiveLogger()]
)

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/avalanche/evaluation/metrics/gpu_usage.py:56: UserWarning: Your system has no GPU!
  warnings.warn("Your system has no GPU!")
/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/avalanche/evaluation/metrics/gpu_usage.py:56: UserWarning: Your system has no GPU!
  warnings.warn("Your system has no GPU!")


## Plugin defitinitions

In [10]:
model_plugins = [WeightStoragePlugin()]

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [11]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.0005)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = GEM(
    model,
    optimizer,
    criterion,
    train_epochs=5,
    train_mb_size=128,
    plugins=model_plugins,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
    **best_hyperparameters
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [12]:
for experience in tqdm(train_stream):
    strategy.train(experience)
    strategy.eval(test_stream)

  0%|          | 0/20 [00:00<?, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:06<00:00, 71.01it/s]
Epoch 0 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 107.9000
	Loss_MB/train_phase/train_stream/Task000 = 1.0047
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0001
	Time_Epoch/train_phase/train_stream/Task000 = 6.6069
100%|██████████| 469/469 [00:08<00:00, 56.77it/s]
Epoch 1 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 114.6000
	Loss_MB/train_phase/train_stream/Task000 = 0.8463
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 8.2615
100%|██████████| 469/469 [00:09<00:00, 50.88it/s]
Epoch 2 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 109.0000
	Loss_MB/train_phase/train_stream/Task000 = 0.6619
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 9.2168
100%|██████████| 469/469 [00:10<00:00, 45.35it/s]
Epoch 3 ended.
	CPUUsage_MB/train_phase/train_stre

  5%|▌         | 1/20 [01:35<30:12, 95.38s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 69.7945
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x15f3a8df0>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9110, 0.1253, 0.1379, 0.1450, 0.1350, 0.0995, 0.1301, 0.1112, 0.1039,
         0.1289, 0.0996, 0.1253, 0.1134, 0.1176, 0.0964, 0.1011, 0.0963, 0.1242,
         0.0905, 0.1508],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

 10%|█         | 2/20 [06:02<58:50, 196.15s/it]

...

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 64.7042
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x17e2a6070>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9110, 0.1253, 0.1379, 0.1450, 0.1350, 0.0995, 0.1301, 0.1112, 0.1039,
         0.1289, 0.0996, 0.1253, 0.1134, 0.1176, 0.0964, 0.1011, 0.0963, 0.1242,
         0.0905, 0.1508],
        [0.7428, 0.9273, 0.1314, 0.1319, 0.1684, 0.0840, 0.1060, 0.1014, 0.1174,
         0.1545, 0.1157, 0.1216, 0.1313, 0.1152, 0.1555, 0.0795, 0.0935, 0.1037,
         0.1228, 0.1172],
        [0.6532, 0.8546, 0.9319, 0.1148, 0.1181, 0.1004, 0.0783, 0.1099, 0.1165,
         0.1217, 0.0954, 0.1328, 0.1326, 0.0784, 0.1693, 0.0666, 0.0728, 0.1314,
         0.1844, 0.0998],
        [0.5979, 0.8038, 0.8554, 0.9325, 0.1082, 0.0625, 0.0886, 0.1109, 0.1014,
         0.1076, 0.0713, 0.1428, 0.1396, 0.0977, 0.15

 25%|██▌       | 5/20 [24:03<1:25:08, 340.56s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 65.3615
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x17f16bd60>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9110, 0.1253, 0.1379, 0.1450, 0.1350, 0.0995, 0.1301, 0.1112, 0.1039,
         0.1289, 0.0996, 0.1253, 0.1134, 0.1176, 0.0964, 0.1011, 0.0963, 0.1242,
         0.0905, 0.1508],
        [0.7428, 0.9273, 0.1314, 0.1319, 0.1684, 0.0840, 0.1060, 0.1014, 0.1174,
         0.1545, 0.1157, 0.1216, 0.1313, 0.1152, 0.1555, 0.0795, 0.0935, 0.1037,
         0.1228, 0.1172],
        [0.6532, 0.8546, 0.9319, 0.1148, 0.1181, 0.1004, 0.0783, 0.1099, 0.1165,
         0.1217, 0.0954, 0.1328, 0.1326, 0.0784, 0.1693, 0.0666, 0.0728, 0.1314,
         0.1844, 0.0998],
        [0.5979, 0.8038, 0.8554, 0.9325, 0.1082, 0.0625, 0.0886, 0.1109, 0.1014,
         0.1076, 0.0713, 0.1428, 0.1396, 0.0977, 0.15

 30%|███       | 6/20 [30:53<1:24:57, 364.13s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 64.7669
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x17f8297f0>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9110, 0.1253, 0.1379, 0.1450, 0.1350, 0.0995, 0.1301, 0.1112, 0.1039,
         0.1289, 0.0996, 0.1253, 0.1134, 0.1176, 0.0964, 0.1011, 0.0963, 0.1242,
         0.0905, 0.1508],
        [0.7428, 0.9273, 0.1314, 0.1319, 0.1684, 0.0840, 0.1060, 0.1014, 0.1174,
         0.1545, 0.1157, 0.1216, 0.1313, 0.1152, 0.1555, 0.0795, 0.0935, 0.1037,
         0.1228, 0.1172],
        [0.6532, 0.8546, 0.9319, 0.1148, 0.1181, 0.1004, 0.0783, 0.1099, 0.1165,
         0.1217, 0.0954, 0.1328, 0.1326, 0.0784, 0.1693, 0.0666, 0.0728, 0.1314,
         0.1844, 0.0998],
        [0.5979, 0.8038, 0.8554, 0.9325, 0.1082, 0.0625, 0.0886, 0.1109, 0.1014,
         0.1076, 0.0713, 0.1428, 0.1396, 0.0977, 0.15

# Get metrics

In [ ]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()
class_accuracies: dict[int, list[float]] = dict()
losses: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    class_accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/{i}"
    ][1]
    losses[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Loss_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

In [ ]:
train_cpu_usage = eval_plugin.get_all_metrics()["CPUUsage_MB/train_phase/train_stream/Task000"][1]
train_time_epoch = eval_plugin.get_all_metrics()["Time_Epoch/train_phase/train_stream/Task000"][1]
train_running_time = eval_plugin.get_all_metrics()["RunningTime_Epoch/train_phase/train_stream/Task000"][1]
train_loss_epoch = eval_plugin.get_all_metrics()["Loss_MB/train_phase/train_stream/Task000"][1]

In [ ]:
weights = model_plugins[0].weights

# Store metrics

In [ ]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt,
    "class_accuracies": class_accuracies,
    "losses": losses,
    "train_cpu_usage": train_cpu_usage,
    "train_time_epoch": train_time_epoch,
    "train_running_time": train_running_time,
    "train_loss_epoch": train_loss_epoch,
    "weights": weights
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))